# NOTE

In [ ]:
from pathlib import Path
import pandas as pd
from datetime import datetime

# กำหนดโฟลเดอร์หลัก
base_path = Path(r"\\10.1.1.124\Project")  # เปลี่ยนเป็น path ของคุณ
target = r"D:\My Documents\sutripat.ng\OneDrive - Thainamthip\1. Work\TDWPBI_Project.xlsx"
rows = []

for item in base_path.rglob("*"):
    if item.is_file():
        # โฟลเดอร์ relative path
        folder = str(item.parent.relative_to(base_path))
        if folder == ".":
            folder = ""
            top_folder = ""
        else:
            folder = folder.replace("/", "\\") + "\\"
            # top folder คือโฟลเดอร์แรก
            top_folder = folder.split("\\")[0]

        rows.append({
            "No": len(rows) + 1,
            "Path" : str(item),
            "Project": top_folder,
            "FolderPath" : folder,
            "File": item.name,
            "ExecuteDate": datetime.now()
        })

df = pd.DataFrame(rows)

# แสดงผลโดยไม่แสดง index
display(df)

df.to_excel(target, index=False)

,No,Path,TopFolder,Folder,File,ExecuteDate
0,1,\\10.1.1.124\Project\How to install AccessData...,,,How to install AccessDatabaseEngine.txt,2025-11-18 08:47:14.004240
1,2,\\10.1.1.124\Project\Thumbs.db,,,Thumbs.db,2025-11-18 08:47:14.099493
2,3,\\10.1.1.124\Project\# BACKUP\Fix_Code_2019031...,# BACKUP,# BACKUP\,Fix_Code_20190318.zip,2025-11-18 08:47:14.461415
3,4,\\10.1.1.124\Project\# BACKUP\Project-20190920...,# BACKUP,# BACKUP\,Project-20190920.zip,2025-11-18 08:47:14.479235
4,5,\\10.1.1.124\Project\# BACKUP\Project.zip,# BACKUP,# BACKUP\,Project.zip,2025-11-18 08:47:14.488870
...,...,...,...,...,...,...
14960,14961,\\10.1.1.124\Project\ZTEMP\PIC\Design C\l_o-da...,ZTEMP,ZTEMP\PIC\Design C\,l_o-dashboard-CC-C-Icon-HRC.png,2025-11-18 08:52:21.507585
14961,14962,\\10.1.1.124\Project\ZTEMP\PIC\Design C\l_o-da...,ZTEMP,ZTEMP\PIC\Design C\,l_o-dashboard-CC-C-Icon-MK.png,2025-11-18 08:52:21.520584
14962,14963,\\10.1.1.124\Project\ZTEMP\PIC\Design C\l_o-da...,ZTEMP,ZTEMP\PIC\Design C\,l_o-dashboard-CC-C-Icon-SPHP.png,2025-11-18 08:52:21.533656
14963,14964,\\10.1.1.124\Project\ZTEMP\PIC\Design C\l_o-da...,ZTEMP,ZTEMP\PIC\Design C\,l_o-dashboard-CC-C-Icon-TT.png,2025-11-18 08:52:21.546970


In [5]:
from datetime import datetime

print(datetime.now())

2025-11-18 08:43:36.522342


In [2]:
import pandas as pd

rowin = []
data = {"No":1, "Name":"Sutripa", "Year":2}

rowin.append(data)
print(rowin)

[{'No': 1, 'Name': 'Sutripa', 'Year': 2}]
